#### Business Analytics FHDW 2025

## Aufgabe

Wir betrachten noch einmal den Datensatz *FlightDelays.csv*.

1. Interaktiv: Was sind hier sinnvolle Prädiktoren für die Verspätung von Flügen?

2. Bereiten Sie den Datensatz für die Generierung von Entscheidungsbäumen vor: Wandeln Sie die erforderlichen Variablen in kategorische um, erzeugen Sie Dummy-Variablen und teilen Sie 60% Trainingsdaten und 40% Validierungsdaten auf.

3. Generieren Sie einen Entscheidungsbaum mit einer maximalen Tiefe von 8 zur Vorhersage von Verspätungen und plotten Sie ihn. Lassen sich daraus einfache praktische Regeln für die Vorhersage von Verspätungen ableiten?

4. Beurteilen Sie die Qualität der Vorhersage und vergleichen Sie diese mit den Ergebnissen der naiven Bayes-Klassifikation (Konfusionsmatrizen und Kreuzvalidierung). Verändern Sie die maximale Tiefe des Baums und beurteilen Sie die Auswirkungen auf die Prädiktionen (besser, schlechter).

## Zu 1

*Day of month* ist für eine kategorische Variable zu granular und ist daher für eine Vorhersage nicht sehr hilfreich. *Departing time* bezieht sich, anders als die *CRS departing time* (*computerized reservation system*) unmittelbar auf eine Verspätung und gehört daher zu dem, was wir voraussagen wollen. *Flight date* bezieht sich auf einen bestimmten Zeitraum, dessen Eigenschaften nicht repräsentativ sein müssen. *Weather* bezieht sich als Ursache auch direkt auf eine Verspätung. Also berücksichtigen wir die nicht im Modell.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from dmba import plotDecisionTree, classificationSummary

delays_df = pd.read_csv('./Daten/FlightDelays.csv')
delays_df = delays_df.drop(columns = ['DAY_OF_MONTH', 'DEP_TIME', 'FL_DATE', 'Weather'])  
delays_df

## Zu 2

In [ ]:
delays_df.CARRIER = delays_df.DAY_WEEK.astype('category')
delays_df.DAY_WEEK = delays_df.DAY_WEEK.astype('category')
delays_df.DEST = delays_df.DEST.astype('category')
delays_df.FL_NUM = delays_df.FL_NUM.astype('category')
delays_df.ORIGIN = delays_df.ORIGIN.astype('category')
delays_df.TAIL_NUM = delays_df.TAIL_NUM.astype('category')
delays_df['Flight Status'] = delays_df['Flight Status'].astype('category')

delays_df.CRS_DEP_TIME = [round(t/100) for t in delays_df.CRS_DEP_TIME]
delays_df.CRS_DEP_TIME = delays_df.CRS_DEP_TIME.astype('category')

X = pd.get_dummies(delays_df[delays_df.columns.difference(['Flight Status'])])
y = delays_df['Flight Status']

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
X

## Zu 3

In [ ]:
class_tree = DecisionTreeClassifier(random_state=1, max_depth=8)
class_tree.fit(train_X, train_y)
plotDecisionTree(class_tree, feature_names=train_X.columns,
                 class_names=class_tree.classes_, rotate=True)

Es gibt laut der oberen Ebenen des Baums stärkere Einflüsse bestimmter Ziele, Flüge, Flieger (tail numbers) und Abflugzeiten.

## Zu 4

Parameter ändern unter 3, dann auswerten:

In [ ]:
classificationSummary(train_y, class_tree.predict(train_X))
print()
classificationSummary(valid_y, class_tree.predict(valid_X))
print()
scores = cross_val_score(class_tree, train_X, train_y, cv=5)
print('Genauigkeit über jede Stichprobe: ', [f'{acc:.3f}' for acc in scores])

## Einsatz von Random Forests und Boosting

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

random_forest = RandomForestClassifier(n_estimators=500, random_state=1)
random_forest.fit(train_X, train_y)

classificationSummary(valid_y, random_forest.predict(valid_X))

In [ ]:
boosted = GradientBoostingClassifier(n_estimators=1, random_state=1)
boosted.fit(train_X, train_y)
classificationSummary(valid_y, boosted.predict(valid_X))

In [ ]:
print(len(delays_df[(delays_df['Flight Status']=='delayed')]))
print(len(delays_df[(delays_df['Flight Status']=='delayed')])/len(delays_df))
print(len(delays_df[(delays_df['Flight Status']=='ontime')]))